---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [8]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [9]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [10]:
def get_list_of_university_towns0():
#     '''Returns a DataFrame of towns and the states they are in from the 
#     university_towns.txt list. The format of the DataFrame should be:
#     DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
#     columns=["State", "RegionName"]  )
    
#     The following cleaning needs to be done:

#     1. For "State", removing characters from "[" to the end.
#     2. For "RegionName", when applicable, removing every character from " (" to the end.
#     3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    df2=pd.read_table('university_towns.txt',sep='\t',header=None,names=['RegionName'])
    df2['State']=df2[df2['RegionName'].str.contains('edit')]
    df2['State'].fillna(method='ffill',inplace=True)
#     df2['check']=df2['State']== df2['RegionName']
#     df2.drop(df2.loc[df2['check']== True].index,inplace=True)
#     del df2['check']
    df2['State'] = df2['State'].str.replace("\[.*?\]",'')
    df2['RegionName']=df2['RegionName'].str.replace("\(.*?\)",'').str.replace("\[.*?\]",'').str.rstrip()
    df2 = df2.loc[df2.RegionName!=df2.State, ['State','RegionName']]
    return df2
# len(df2['RegionName'].unique())
# get_list_of_university_towns0()

In [11]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame is:
    DataFrame( [ ["Michigan","Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State","RegionName"]  )'''
    uni = pd.read_table('university_towns.txt', sep='\t', header=None, names=['RegionName'])
    uni['State'] = [x.split('[')[0] if 'edit' in x else np.nan for x in uni['RegionName']]
    uni['State'].fillna(method='ffill', inplace=True)
    uni['RegionName'] = uni['RegionName'].str.split('(',expand=True)[0].str.split('[', expand=True)[0].str.rstrip()
    uni = uni.loc[uni.RegionName!=uni.State, ['State','RegionName']]
    return uni


In [12]:
df3=pd.read_excel('gdplev.xls',skiprows=6)
df3= df3.rename(columns={'Unnamed: 4':'Year','Unnamed: 6':'Gdp'})
GDP=df3[['Year','Gdp']].copy()
GDP.dropna(axis=0, how='all',inplace=True)
GDP.drop(GDP[0:212].index,inplace=True)
    

In [13]:
GDP[GDP['Year'].str.contains('2008','2009')]

,Year,Gdp
245,2008q1,14889.5
246,2008q2,14963.4
247,2008q3,14891.6
248,2008q4,14577.0


In [14]:
# def get_recession_start():
# #     '''Returns the year and quarter of the recession start time as a 
# #     string value in a format such as 2005q3'''
#     list_Gdp= GDP['Gdp'].tolist()
#     start_list=[]
#     for i in range(len(list_Gdp)-2):
#         if(GDP.iloc[i+1][1] < GDP.iloc[i][1]) and (GDP.iloc[i+2][1] < GDP.iloc[i+1][1]):
#             start_list.append(GDP.iloc[i][0])
            
#     return start_list[1]
# get_recession_start()
def get_recession_start():
#     '''Returns the year and quarter of the recession start time as a 
#     string value in a format such as 2005q3'''
    GDP['recession'] = GDP['Gdp'].diff(periods=1)
    GDP.reset_index(drop=True, inplace=True)
    GDP['start_recession']=False

    GDP.loc[GDP['recession']>0]
    for i in range(len(GDP)-2):
        if(GDP.loc[i,'recession']<0) and (GDP.loc[i+1,'recession']<0):
            GDP.loc[i,'start_recession']=True
    start = GDP.loc[GDP[GDP.start_recession==True].index.min(),'Year']
    return start
get_recession_start()

'2008q3'

In [15]:
# def get_recession_end():
# #     '''Returns the year and quarter of the recession end time as a 
# #     string value in a format such as 2005q3'''
#     GDP_copy=GDP.copy()
#     GDP_copy['Gdp']=GDP_copy['Gdp'].astype(float)
#     GDP_copy.drop(GDP_copy[0:35].index,inplace=True)
    
#     list_Gdp= GDP_copy['Gdp'].tolist()
#     end_list=[]
#     for i in range(len(list_Gdp)-2):
#         if(GDP_copy.iloc[i][1] < GDP_copy.iloc[i+1][1]) and (GDP_copy.iloc[i+1][1] < GDP_copy.iloc[i+2][1]):
#             end_list.append(GDP_copy.iloc[i][0])       
#     return end_list[2]
# get_recession_end()
def get_recession_end():
    GDP_copy=GDP.copy()
    GDP_copy['Gdp']=GDP_copy['Gdp'].astype(float)
    GDP_copy.drop(GDP_copy[0:35].index,inplace=True)

    GDP_copy['recession'] = GDP_copy['Gdp'].diff(periods=1)
    GDP_copy.reset_index(drop=True, inplace=True)
    GDP_copy['end_recession']=False

    for i in range(len(GDP_copy)-2):
        if(GDP_copy.loc[i,'recession']>0) and (GDP_copy.loc[i+1,'recession']>0):
            GDP_copy.loc[i,'end_recession']=True
    end = GDP_copy.loc[GDP_copy[GDP_copy.end_recession==True].index[1],'Year']

    return end
get_recession_end()

'2009q4'

In [16]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    GDP['recession'] = GDP['Gdp'].diff(periods=1)
    GDP.reset_index(drop=True, inplace=True)
    GDP_copy0=GDP.copy()
#     GDP_copy0=GDP_copy0.set_index('Year')
    #start->2008q3
    #end->2009q4
    start=GDP_copy0[GDP_copy0['Year']=='2008q3'].index
    end=GDP_copy0[GDP_copy0['Year']=='2009q4'].index
    return GDP_copy0.loc[34:37].max()['Year']

get_recession_bottom()

'2009q2'

In [17]:
# def get_recession_bottom():
#     '''Returns the year and quarter of the recession bottom time as a 
#     string value in a format such as 2005q3'''
#     import pandas as pd
#     gdp_usa = pd.read_excel('gdplev.xls', skiprows=5, usecols=[4,6], names=[
#                 'Year&Quarter','GDP_chained_2009_dollars'])
#     gdp_usa.dropna(axis=0, how='all', inplace=True)
#     gdp_usa['year'] = [int(x[0:4]) for x in gdp_usa['Year&Quarter']]
#     gdp_usa['quarter'] = [x[4:] for x in gdp_usa['Year&Quarter']]
#     gdp_usa = gdp_usa[gdp_usa['year']>=2000]
#     gdp_usa['recession'] = gdp_usa['GDP_chained_2009_dollars'].diff(periods=1)
#     gdp_usa.reset_index(drop=True, inplace=True)
#     gdp_usa['recessionStart'] = False
#     gdp_usa['recessionEnd'] = False
#     for i in range(1,len(gdp_usa)-1):
#         if (gdp_usa.loc[i,'recession']<0)&(gdp_usa.loc[i+1,'recession']<0):
#             gdp_usa.loc[i,'recessionStart'] = True
#     for i in range(3,len(gdp_usa)):
#         if ((gdp_usa.loc[i-3,'recession']<0)&(gdp_usa.loc[i-2,'recession']<0)&
#             (gdp_usa.loc[i-1,'recession']>0)&(gdp_usa.loc[i,'recession']>0)):
#             gdp_usa.loc[i,'recessionEnd'] = True  
#     recessionStart = gdp_usa[gdp_usa['recessionStart']==True].index.min()
#     recessionEnd = gdp_usa[gdp_usa['recessionEnd']==True].index.min()
#     df_rec = gdp_usa.ix[recessionStart:recessionEnd,:]
#     ans = df_rec.loc[df_rec['GDP_chained_2009_dollars']==df_rec['GDP_chained_2009_dollars'].min(),'Year&Quarter'] 
#     return df_rec
# get_recession_bottom()

In [18]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    house=pd.read_csv('City_Zhvi_AllHomes.csv')
    col_lst=house.loc[:,'1996-04':'1999-12']

    house.drop(col_lst,axis=1,inplace=True)

    house.columns = (house.columns.str.replace('-01', 'q1').str.replace('-02', 'q1')
                         .str.replace('-03', 'q1').str.replace('-04', 'q2').str.replace('-05', 'q2')
                         .str.replace('-06', 'q2').str.replace('-07', 'q3').str.replace('-08', 'q3')
                         .str.replace('-09', 'q3').str.replace('-10', 'q4').str.replace('-11', 'q4')
                         .str.replace('-12', 'q4'))
    house=house.set_index(['State','RegionName'])
    col_lst2=house.loc[:,'RegionID':'SizeRank']
    house.drop(col_lst2,axis=1,inplace=True)
    house = house.groupby(house.columns, axis=1).mean()


    return house


In [19]:
def convert_housing_data_to_quarters0():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. The resulting dataframe should have 67 
    columns, and 10,730 rows.
    '''
    all_homes = pd.read_csv('City_Zhvi_AllHomes.csv')
    all_homes['State'] = all_homes['State'].map(states)
    all_homes.drop(all_homes.filter(regex='19').columns, axis=1, inplace=True)
    all_homes.columns = (all_homes.columns.str.replace('-01', 'q1').str.replace('-02', 'q1')
                         .str.replace('-03', 'q1').str.replace('-04', 'q2').str.replace('-05', 'q2')
                         .str.replace('-06', 'q2').str.replace('-07', 'q3').str.replace('-08', 'q3')
                         .str.replace('-09', 'q3').str.replace('-10', 'q4').str.replace('-11', 'q4')
                         .str.replace('-12', 'q4'))
    all_homes = (all_homes.set_index(['State','RegionName'], drop=True)
               .drop(['RegionID','Metro','CountyName','SizeRank'],axis=1))          
    all_homes = all_homes.groupby(by=all_homes.columns, axis=1).mean() 
    
    return all_homes


In [52]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    #recession
    house =convert_housing_data_to_quarters()
    house_copy=house.xs(['2008q3','2008q4','2009q1','2009q2'], axis=1)
    uni = get_list_of_university_towns().set_index(['State', 'RegionName'])
#     house_uni=house_copy.merge(uni,left_index=True, right_index=True,how='inner')
    return house_copy

